In [1]:
import pandas as pd
import numpy as np
import pickle
from script import *
from sarimax_script import *
from garch_script import *
from appliances_script import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
plt.rc('figure', figsize=(18.0, 6.0))
pd.set_option('display.max_columns', None) 

In [87]:
df_545 = pickle.load(open('pickle_files/df_545.p','rb'))

In [1]:
#function to find charge peaks
def charge_peak(row):
    if row['car1'] > 0.8:
        return 1
    else:
        return 0

In [94]:
#function to find mean car charge at time
def mean_car_charge(data, start, end):
    car_charge = {}
    for index in data.Time_Index.unique():
        car_charge[index] = np.mean(data[data.Time_Index==index].car1)
    
    return car_charge

In [88]:
start_train, end_train, start_test, end_test = find_dates(545, 15, 90)

In [89]:
df_545.index = pd.to_datetime(df_545.apply(strip_tz, axis=1))

In [90]:
df_545_car = df_545.loc[start_train:end_test,['localhour','car1']]

In [92]:
df_545_car['Car_Charge'] = df_545_car.apply(charge_peak, axis=1)
df_545_car['Day_of_Week'] = df_545_car.index.weekday_name
df_545_car['Hour'] = df_545_car.index.hour

In [93]:
df_545_car['Time_Index'] = df_545_car['Day_of_Week']+ df_545_car['Hour'].astype(str)

In [78]:
#find mean of each time index
car_charge = {}

for index in df_545_car.Time_Index.unique():
    car_charge[index] = np.mean(df_545_car[df_545_car.Time_Index ==index].car1)

In [48]:
df_545_car

,localhour,car1,Car_Charge,Day_of_Week,Hour,Time_Index
2016-08-02 00:00:00,2016-08-02 00:00:00-05:00,0.000000,0,Tuesday,0,Tuesday0
2016-08-02 01:00:00,2016-08-02 01:00:00-05:00,0.000000,0,Tuesday,1,Tuesday1
2016-08-02 02:00:00,2016-08-02 02:00:00-05:00,0.000000,0,Tuesday,2,Tuesday2
2016-08-02 03:00:00,2016-08-02 03:00:00-05:00,0.114117,0,Tuesday,3,Tuesday3
2016-08-02 04:00:00,2016-08-02 04:00:00-05:00,0.000000,0,Tuesday,4,Tuesday4
2016-08-02 05:00:00,2016-08-02 05:00:00-05:00,0.000000,0,Tuesday,5,Tuesday5
2016-08-02 06:00:00,2016-08-02 06:00:00-05:00,0.000367,0,Tuesday,6,Tuesday6
2016-08-02 07:00:00,2016-08-02 07:00:00-05:00,0.000000,0,Tuesday,7,Tuesday7
2016-08-02 08:00:00,2016-08-02 08:00:00-05:00,0.000500,0,Tuesday,8,Tuesday8
2016-08-02 09:00:00,2016-08-02 09:00:00-05:00,0.000100,0,Tuesday,9,Tuesday9


In [49]:
pickle.dump(df_545_car,open('car_charge_exog.p','wb'))